<a href="https://colab.research.google.com/github/albert-reds/Diagnostico-de-Condiciones-Medicas-en-Rayos-X-de-Torax/blob/main/Diagnostico_DeepLearning_Torax_GAGR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NIH ChestX-ray14 — Comparativa (DenseNet121, EfficientNet‑B2, Swin‑T) + Inferencia con Gradio

Notebook **final y limpio** para Google Colab. Incluye:
- Montaje de Google Drive
- Indexación robusta en `images_0xx/**/images/*.(png|jpg|jpeg)`
- Creación de `train.csv`, `val.csv`, `test.csv` desde los archivos oficiales
- Dataset/Dataloaders basados en `FILE_MAP` (sin rutas frágiles)
- Entrenamiento/evaluación de **3 modelos** (configurable)
- Interfaz **Gradio** para probar imágenes nuevas (+ Grad‑CAM para CNNs)

**Antes de empezar:** `Entorno de ejecución → Cambiar tipo de entorno → Acelerador de hardware: GPU`.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

DATA_DIR = "/content/drive/MyDrive/datasets/NIH-ChestXray"

import os, platform, torch
assert os.path.isdir(DATA_DIR), f"❌ No existe DATA_DIR: {DATA_DIR}"
print("✅ DATA_DIR:", DATA_DIR)
print("Python:", platform.python_version())
print("Torch :", torch.__version__)
print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


Mounted at /content/drive
✅ DATA_DIR: /content/drive/MyDrive/datasets/NIH-ChestXray
Python: 3.12.11
Torch : 2.8.0+cu126
CUDA disponible: False


In [2]:
import os
DATA_CSV = os.path.join(DATA_DIR, "Data_Entry_2017.csv")
TRAIN_VAL_LIST = os.path.join(DATA_DIR, "train_val_list.txt")
TEST_LIST = os.path.join(DATA_DIR, "test_list.txt")
for p in [DATA_CSV, TRAIN_VAL_LIST, TEST_LIST]:
    assert os.path.exists(p), f"❌ Falta {os.path.basename(p)} en {DATA_DIR}"
print("✅ Archivos oficiales detectados.")


✅ Archivos oficiales detectados.


In [3]:
from pathlib import Path
def build_file_map(data_dir):
    file_map = {}
    roots = sorted([d for d in Path(data_dir).glob("images_*") if d.is_dir()])
    assert roots, "❌ No se encontraron carpetas images_0xx en DATA_DIR"
    exts = {".png",".jpg",".jpeg"}
    total=0
    for root in roots:
        for p in root.rglob("*"):
            if p.is_file() and p.suffix.lower() in exts:
                file_map[p.name] = str(p); total += 1
    print(f"   ↳ Escaneadas {len(roots)} carpetas 'images_0xx', archivos contados: {total}")
    return file_map

FILE_MAP = build_file_map(DATA_DIR)
print("✅ Imágenes indexadas (filename → ruta):", len(FILE_MAP))
for i,(k,v) in enumerate(FILE_MAP.items()):
    if i>=3: break
    print("  ", k, "->", v)
assert len(FILE_MAP) > 0, "❌ No se indexaron imágenes. Revisa la estructura 'images_0xx/**/images/'."


   ↳ Escaneadas 1 carpetas 'images_0xx', archivos contados: 4999
✅ Imágenes indexadas (filename → ruta): 4999
   00001107_000.png -> /content/drive/MyDrive/datasets/NIH-ChestXray/images_001/images/00001107_000.png
   00001103_002.png -> /content/drive/MyDrive/datasets/NIH-ChestXray/images_001/images/00001103_002.png
   00001089_002.png -> /content/drive/MyDrive/datasets/NIH-ChestXray/images_001/images/00001089_002.png


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

df_all = pd.read_csv(DATA_CSV)
with open(TRAIN_VAL_LIST, "r") as f:
    train_val_files = [line.strip().split()[0] for line in f if line.strip()]
with open(TEST_LIST, "r") as f:
    test_files = [line.strip().split()[0] for line in f if line.strip()]

df_trainval_raw = df_all[df_all["Image Index"].isin(train_val_files)].copy()
df_test_raw     = df_all[df_all["Image Index"].isin(test_files)].copy()

def keep_existing(df):
    return df[df["Image Index"].map(lambda x: x in FILE_MAP)].copy()

df_trainval = keep_existing(df_trainval_raw)
df_test     = keep_existing(df_test_raw)
print(f"📑 train_val con imágenes: {len(df_trainval)} | test con imágenes: {len(df_test)}")
assert len(df_trainval)>0, "❌ train_val quedó vacío tras filtrar por imágenes existentes."

def safe_split(df, test_size=0.10, seed=42):
    y = df["Finding Labels"].astype(str)
    vc = y.value_counts()
    can_stratify = (len(vc)>=2) and (vc.min()>=2) and (len(df)*test_size>=len(vc))
    if can_stratify:
        return train_test_split(df, test_size=test_size, random_state=seed, stratify=y)
    else:
        print("⚠️ Pocas muestras/clases → split sin estratificar.")
        return train_test_split(df, test_size=test_size, random_state=seed, shuffle=True)

df_train, df_val = safe_split(df_trainval, test_size=0.10, seed=42)
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
VAL_CSV   = os.path.join(DATA_DIR, "val.csv")
TEST_CSV  = os.path.join(DATA_DIR, "test.csv")
df_train.to_csv(TRAIN_CSV, index=False)
df_val.to_csv(VAL_CSV, index=False)
df_test.to_csv(TEST_CSV, index=False)
print("🎯 Guardados:")
print("  train.csv:", len(df_train), "→", TRAIN_CSV)
print("  val.csv  :", len(df_val),   "→", VAL_CSV)
print("  test.csv :", len(df_test),  "→", TEST_CSV)


📑 train_val con imágenes: 4032 | test con imágenes: 967
⚠️ Pocas muestras/clases → split sin estratificar.
🎯 Guardados:
  train.csv: 3628 → /content/drive/MyDrive/datasets/NIH-ChestXray/train.csv
  val.csv  : 404 → /content/drive/MyDrive/datasets/NIH-ChestXray/val.csv
  test.csv : 967 → /content/drive/MyDrive/datasets/NIH-ChestXray/test.csv


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

IMAGENET_MEAN=[0.485,0.456,0.406]; IMAGENET_STD=[0.229,0.224,0.225]
IMG_SIZE=224
train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])
eval_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

class ChestXrayDatasetFM(Dataset):
    def __init__(self, csv_file, file_map, transform):
        import pandas as pd
        self.df = pd.read_csv(csv_file)
        self.file_map = file_map
        self.tfm = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        fname = r["Image Index"]
        p = self.file_map.get(fname)
        if p is None:
            raise FileNotFoundError(f"{fname} no hallado en FILE_MAP")
        img = Image.open(p).convert("RGB")
        x = self.tfm(img)
        y = 0 if str(r["Finding Labels"]).strip()=="No Finding" else 1
        return x, torch.tensor(y, dtype=torch.long)

BATCH=16
NUM_WORKERS = 2 if torch.cuda.is_available() else 0
try:
    tmp_dl = DataLoader(ChestXrayDatasetFM(TRAIN_CSV, FILE_MAP, train_tfms), batch_size=BATCH, shuffle=True, num_workers=NUM_WORKERS, pin_memory=torch.cuda.is_available())
    xb, yb = next(iter(tmp_dl))
    print("✅ Dataloader OK. Batch ejemplo:", xb.shape, yb.shape, "| etiquetas:", yb.unique().tolist())
except Exception as e:
    print("⚠️ Prueba de Dataloader falló:", e)


In [6]:
import torch.nn as nn
from sklearn.metrics import accuracy_score, f1_score
from torchvision.models import (
    densenet121, DenseNet121_Weights,
    efficientnet_b2, EfficientNet_B2_Weights,
    swin_t, Swin_T_Weights
)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_PRETRAINED = True
EPOCHS = 1
BATCH = 16
LR = 1e-4

def build_densenet(num_classes=2):
    w = DenseNet121_Weights.IMAGENET1K_V1 if USE_PRETRAINED else None
    m = densenet121(weights=w)
    m.classifier = nn.Linear(m.classifier.in_features, num_classes)
    return m

def build_efficientnet(num_classes=2):
    w = EfficientNet_B2_Weights.IMAGENET1K_V1 if USE_PRETRAINED else None
    m = efficientnet_b2(weights=w)
    m.classifier[1] = nn.Linear(m.classifier[1].in_features, num_classes)
    return m

def build_swin(num_classes=2):
    w = Swin_T_Weights.IMAGENET1K_V1 if USE_PRETRAINED else None
    m = swin_t(weights=w)
    m.head = nn.Linear(m.head.in_features, num_classes)
    return m

MODELS = {
    "densenet121": build_densenet,
    "efficientnet_b2": build_efficientnet,
    "swin_t": build_swin,
}

def train_eval_model(model_fn, train_csv, val_csv, test_csv, file_map, epochs=EPOCHS, batch_size=BATCH, lr=LR):
    ds_tr = ChestXrayDatasetFM(train_csv, file_map, train_tfms)
    ds_va = ChestXrayDatasetFM(val_csv,   file_map, eval_tfms)
    ds_te = ChestXrayDatasetFM(test_csv,  file_map, eval_tfms)

    nw = 2 if torch.cuda.is_available() else 0
    dl_tr = DataLoader(ds_tr, batch_size=batch_size, shuffle=True,  num_workers=nw, pin_memory=torch.cuda.is_available())
    dl_va = DataLoader(ds_va, batch_size=batch_size, shuffle=False, num_workers=nw)
    dl_te = DataLoader(ds_te, batch_size=batch_size, shuffle=False, num_workers=nw)

    model = model_fn().to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    for ep in range(epochs):
        model.train()
        for x,y in dl_tr:
            x,y = x.to(DEVICE), y.to(DEVICE)
            opt.zero_grad(); loss = loss_fn(model(x), y); loss.backward(); opt.step()

    def eval_dl(dl):
        y_true, y_pred = [], []
        model.eval()
        with torch.no_grad():
            for x,y in dl:
                x,y = x.to(DEVICE), y.to(DEVICE)
                preds = model(x).argmax(1)
                y_true += y.cpu().tolist(); y_pred += preds.cpu().tolist()
        return accuracy_score(y_true,y_pred), f1_score(y_true,y_pred, average="macro")

    tr_acc, tr_f1 = eval_dl(dl_tr)
    va_acc, va_f1 = eval_dl(dl_va)
    te_acc, te_f1 = eval_dl(dl_te)
    return {"train_acc":tr_acc,"train_f1":tr_f1,"val_acc":va_acc,"val_f1":va_f1,"test_acc":te_acc,"test_f1":te_f1}


In [7]:
import pandas as pd
results = []
for name, fn in MODELS.items():
    print(f"=== Entrenando {name} ===")
    mets = train_eval_model(fn, TRAIN_CSV, VAL_CSV, TEST_CSV, FILE_MAP)
    results.append({"model":name, **mets})
df = pd.DataFrame(results)
display(df)
OUT_CSV = os.path.join(DATA_DIR, "resultados_3_modelos.csv")
df.to_csv(OUT_CSV, index=False)
print("📄 Guardado:", OUT_CSV)


=== Entrenando densenet121 ===
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


100%|██████████| 30.8M/30.8M [00:00<00:00, 60.7MB/s]


=== Entrenando efficientnet_b2 ===
Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth


100%|██████████| 35.2M/35.2M [00:00<00:00, 115MB/s]


=== Entrenando swin_t ===
Downloading: "https://download.pytorch.org/models/swin_t-704ceda3.pth" to /root/.cache/torch/hub/checkpoints/swin_t-704ceda3.pth


100%|██████████| 108M/108M [00:01<00:00, 93.7MB/s]


,model,train_acc,train_f1,val_acc,val_f1,test_acc,test_f1
0,densenet121,0.745039,0.721487,0.678218,0.662190,0.599793,0.550578
1,efficientnet_b2,0.744763,0.719027,0.698020,0.672879,0.578077,0.543255
2,swin_t,0.506615,0.476805,0.475248,0.442439,0.662875,0.431382


📄 Guardado: /content/drive/MyDrive/datasets/NIH-ChestXray/resultados_3_modelos.csv


In [ ]:
!pip -q install gradio==4.44.0
import gradio as gr
from torchvision import transforms
from PIL import Image
import numpy as np, os

CLASS_NAMES = [
    "Atelectasis","Cardiomegaly","Effusion","Infiltration","Mass","Nodule",
    "Pneumonia","Pneumothorax","Consolidation","Edema","Emphysema","Fibrosis",
    "Pleural_Thickening","Hernia"
]
infer_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])
MODEL_FACTORY = {
    "densenet121": build_densenet,
    "efficientnet_b2": build_efficientnet,
    "swin_t": build_swin,
}
def load_model_for_inference(model_name: str, num_classes: int = 14, ckpt_path: str = ""):
    model = MODEL_FACTORY[model_name](num_classes=num_classes).to(DEVICE).eval()
    if ckpt_path and os.path.exists(ckpt_path):
        state = torch.load(ckpt_path, map_location=DEVICE)
        if isinstance(state, dict) and "state_dict" in state: state = state["state_dict"]
        try:
            model.load_state_dict(state, strict=False)
        except Exception:
            new_state = {}
            for k,v in (state.items() if isinstance(state, dict) else []): new_state[k.replace("module.","")] = v
            model.load_state_dict(new_state, strict=False)
    return model
def gradcam_quick(model: nn.Module, pil_img: Image.Image):
    import torch.nn.functional as F, cv2
    last_conv = None
    for m in model.modules():
        if isinstance(m, nn.Conv2d): last_conv = m
    if last_conv is None: return None
    activations, gradients = [], []
    def fwd_hook(_, __, out): activations.append(out.detach())
    def bwd_hook(_, grad_in, grad_out): gradients.append(grad_out[0].detach())
    h1 = last_conv.register_forward_hook(fwd_hook)
    h2 = last_conv.register_full_backward_hook(bwd_hook)
    model.eval()
    x = infer_tfms(pil_img).unsqueeze(0).to(DEVICE); x.requires_grad_(True)
    out = model(x); score = out[0].max(); model.zero_grad(set_to_none=True); score.backward()
    act = activations[-1][0]; grad = gradients[-1][0]
    w = grad.mean(dim=(1,2), keepdim=True); cam = (w*act).sum(dim=0); cam = F.relu(cam); cam = cam/(cam.max()+1e-6)
    cam = cam.cpu().numpy()
    import cv2, numpy as np
    img_np = np.array(pil_img.convert("RGB"))
    cam_resized = cv2.resize(cam, (img_np.shape[1], img_np.shape[0]))
    heatmap = cv2.applyColorMap(np.uint8(255*cam_resized), cv2.COLORMAP_JET)
    overlay = np.uint8(0.4*heatmap + 0.6*img_np)
    h1.remove(); h2.remove(); from PIL import Image as PImage
    return PImage.fromarray(overlay)
import torch
@torch.no_grad()
def predict_one(img: Image.Image, model_name: str, ckpt_path: str, topk: int, threshold: float):
    model = load_model_for_inference(model_name, num_classes=14, ckpt_path=ckpt_path)
    x = infer_tfms(img).unsqueeze(0).to(DEVICE)
    logits = model(x)
    import numpy as np
    probs = torch.sigmoid(logits)[0].cpu().numpy()
    idxs = np.argsort(-probs)[:topk]
    top_rows = [(CLASS_NAMES[i], float(probs[i])) for i in idxs]
    above = [(CLASS_NAMES[i], float(probs[i])) for i in range(len(CLASS_NAMES)) if probs[i] >= threshold]
    above = sorted(above, key=lambda t: -t[1])
    cam_img = None
    if model_name in ("densenet121","efficientnet_b2"):
        try: cam_img = gradcam_quick(model, img)
        except Exception: cam_img = None
    return top_rows, above, cam_img
with gr.Blocks(title="ChestX-ray14 Inference") as demo:
    gr.Markdown("## 🩻 ChestX‑ray14 – Prueba con imágenes nuevas")
    with gr.Row():
        with gr.Column(scale=1):
            img_in = gr.Image(type="pil", label="Sube radiografía (PNG/JPG)")
            model_dd = gr.Dropdown(choices=list(MODEL_FACTORY.keys()), value="densenet121", label="Modelo")
            ckpt_tb = gr.Textbox(value="", label="Ruta de checkpoint (.pth/.pt) en Drive (opcional)")
            topk_sl = gr.Slider(1, 14, value=5, step=1, label="Top‑k")
            thr_sl  = gr.Slider(0.05, 0.95, value=0.5, step=0.05, label="Umbral multi‑label")
            btn = gr.Button("Predecir", variant="primary")
        with gr.Column(scale=1):
            out_top = gr.Dataframe(headers=["Clase","Probabilidad"], label="Top‑k", interactive=False)
            out_thr = gr.Dataframe(headers=["Clase","Probabilidad"], label="≥ umbral", interactive=False)
            out_cam = gr.Image(type="pil", label="Grad‑CAM (solo CNNs)")
    btn.click(predict_one, inputs=[img_in, model_dd, ckpt_tb, topk_sl, thr_sl], outputs=[out_top, out_thr, out_cam])
demo.launch(debug=False, share=True)
